In [1]:
! pip install kaggle

In [2]:
#uploading kaggle.json file[ i am using json file because its a large data set]
#after uploading kaggle.json file manually
#configuring the path for kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
#APi to fetch the dataset from kaggle
#this is the API command
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:05<00:00, 20.1MB/s]
100% 80.9M/80.9M [00:05<00:00, 16.8MB/s]


In [4]:
#extracting the zip file[dataset]
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip: # r means read
  zip.extractall()
  print("Dataset is extracted - training.1600000.processed.noemotion.csv")


Dataset is extracted - training.1600000.processed.noemotion.csv


In [5]:
#importing the dependencies

import numpy as np
import pandas as pd
import re #regular expression used for pattern matching

from nltk.corpus import stopwords #from natural language tool kit and from corpus module i am importing stopwords [ ]
from nltk.stem.porter import PorterStemmer #PorterStemmer is used to reduce the word to its root word

from sklearn.feature_extraction.text import TfidfVectorizer #to convert textual data to numerical data
from sklearn.model_selection import train_test_split #used to split the data set into training and testing data
from sklearn.linear_model import LogisticRegression #i will be using logistic regression model
from sklearn.metrics import accuracy_score

In [6]:
#downloading the stopwords
import nltk
nltk.download('stopwords')
#stopwords are just english words which are not important to the context

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#printing the stopwords in english
print(stopwords.words('english'))
#the machine learning model do not require these words in processing so i will remove this from the text

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#Data processing[ loading the data from csv file to pandas dataframe ]



In [8]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1' ) #this is the standard encoding method

In [9]:
#checking the number of rows and coloumns
twitter_data.shape
twitter_data.head(8)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it


In [10]:
 #naming the coulumns
 coloumn_names = ['target', 'id', 'date', 'flag', 'user', 'text']
 #now reading the dataset file again
 twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = coloumn_names, encoding = 'ISO-8859-1' ) #this is the standard encoding method

In [11]:
twitter_data.shape
print("The updated data set with coloumn names")
twitter_data.head(9)


The updated data set with coloumn names


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it


**The csv Data set is being loaded in the Dataframe**

In [12]:
#checking if there is any values missing, i can either drop the values or replace the missing values
#counting the number of missing values
twitter_data.isnull().sum() #isnull() is a pandas function to check the null values summing values and i will sum them
# if it shows 0 then all the values are present and there is no misssing values

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [13]:
#checking the distribution of target coloumn [0 = negative, 2 = neutral, 4 = positive]
twitter_data['target'].value_counts() #in the square brackets i have mentioned the coulmn to be checked

,count
target,
0,800000
4,800000


So there is 800000 tweets for the value 0(negative)
and there is 800000 tweets for the value 4(positive)

In [14]:
#convert the Target label from '4' to '1'
twitter_data.replace({'target': {4:1}}, inplace = True) #inplace will store the converter data in the original data frame(in a dictionary)

twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 -> Negative tweet

1 -> Positive tweet

 **Stemming process**

 It is the process of reducing the word to its root word

 ex: [actor, acting,actress, acting] = 'act'

 because my model only understands numerical value, so i will reduce the word to its root word to numercial value

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
  stemmed_content = re.sub('[^a-z A-Z]', ' ', content) #in content i will pass the text coloumn(each tweet in the dataframe) and process all the text, ^ will remove all the characters in the data frame apart from to a-z and A-Z, basically i am trying to remove all the special characters like numbers and @
  stemmed_content = stemmed_content.lower() #converting all the letters to lower case
  stemmed_content = stemmed_content.split() #splitting all the words in this and storing it in the list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # if a particular word is not in the stopwords, then perform the stemmin operation
  stemmed_content = ' '.join(stemmed_content) # joining all the words in a particular tweet

  return stemmed_content

Now applying this function on twitter data frame

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)#new coloumn will be created for stemmed_data using Text value from the dataframe


This stemming operation takes a while because it must go throught 1600000 tweets from the data frame


approx 1 hr

In [ ]:
#a new coloumn "stemmed_content" will be formed now
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
twitter_data.head(9)



,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug,need hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...,loltrish hey long time see ye rain bit bit lol...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it,tatiana k nope


**This is the final Data set I will be using to training my machine learning model**

In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#i need the sentiment analysis only for the tweet, seperating the data and the label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values
print("stemmed_content =  ",X)
print("target =  ",Y)

stemmed_content =   ['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
target =   [0 0 0 ... 1 1 1]


**Splitting the data into TRAINING and TESTING**

In [ ]:
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state = 2)

X_train contains all the *training data tweets*, and corresponding *Target labels* are stored in Y train and X_test contains the test data tweets and Y_test contains the corresponding test target values

X_traing linked with Y_train


X_test linked with Y_test

test size is the amount of data divided into training and testing data, Here i am taking 0.2 that is 20 % of data will be tested and the remaining 80% will be trained


strtify gives equal distribution of "0" and "1" in both the training and testing data


random state 2 will shuffle in a certain manner and divide the training and testing data


In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper'] ['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [ ]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


*Since Machine learning models dont understand text data, therefore i must convert it to numerical value which is also called as* **Feature extraction**



will be using a method called **vectorizer**

In [ ]:
vectorizer = TfidfVectorizer() #this vectorizer will convert text value to numerical value
X_train = vectorizer.fit_transform(X_train) #all the training data will be taken and the vectorizer wil score it from 0 and 1 [positive or negative]

X_test = vectorizer.transform(X_test)

fit_transform  -> fit will *understand* the nature of data ans transform will *convert* textual to numerical data


fit_transform must be used only for training data

transform must be used for testing data


In [ ]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [ ]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

**Training the machine learning model with Logistic Regression**
it is a classification model where i will be classifying the data into 0 negative or 1 positive

In [ ]:
model = LogisticRegression(max_iter = 1000)# max_iter --> maximum number of times the model will go through the data
model.fit(X_train , Y_train) #this is where the model will learn from the data

LogisticRegression(max_iter=1000)

**Model evaluation**


i will be using Accuracy Score, to check the correct prediction

In [ ]:
#accuracy prediction on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction) #accuracy_score is a function which we imported

once the model completes learning from the data, then i will try to make predictions based on x train data


*X_train prediction data based on Y_train true labels*

In [ ]:
print("Accuracy score on the TRAINING DATA  ->  ", training_data_accuracy)

accuracy_score on the TRAINING DATA  ->   0.81018984375


**The Accuracy is 81% for X_train data**

If i use this trained model in other tweet data sets, then in 100 tweet i will get 81 correct prediction and remaining 19 wrong prediction


In [ ]:
#accuracy prediction on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction) #accuracy_score is a function which we imported

In [ ]:
print("The Accuracy score on TEST DATA  ->  ", test_data_accuracy)

The Accuracy score on TEST DATA  ->   0.7780375


**The Accuracy is 77.8% for X_test data**

In my case the accuracy of training data and testing data is close by,


When the trianing data accuracy is more and  testing data accuracy is very less, then its call as **Overfitting**


Overfitting means the model hasn't learned properly

In [ ]:
#saving the trained model
import pickle

In [ ]:
filename = "Twitter_sentiment_trained_model.sav"
pickle.dump(model,open(filename, 'wb')) #wb will write in binary form

**Model Saved**


i saved the trained model using Pickle library in .sav extention

**Using the saved model in future prediction**

In [ ]:
#loading the saved model
loaded_model = pickle.load(open("/content/Twitter_sentiment_trained_model.sav", 'rb')) #rb will read the file in binary form

In [ ]:
X_new = X_test[200]
print(Y_test[200]) #200th tweet with true value of 1
predict = loaded_model.predict(X_new)
print(predict)

if (predict[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

1
[1]
positive tweet


In [ ]:
X_new = X_test[3]
print(Y_test[3]) #3rd tweet with true value of 0
predict = loaded_model.predict(X_new)
print(predict)

if (predict[0] == 0):
  print("negative tweet")

else:
  print("positive tweet")

0
[0]
negative tweet


In [ ]:
def Pred(n):
  X_new = X_test[n]
  print(Y_test[n]) #nth tweet with true value of 1
  predict = loaded_model.predict(X_new)
  print(predict)

  if (predict[0] == 0):
    print("negative tweet")

  else:
    print("positive tweet")


  return

In [ ]:
Pred(3)

0
[0]
negative tweet


*all the steps performed* --

1) import the kaggle library, using PI i extracted the dataset from the website in the form of json, because the dataset was large

2)its stored in the .zip, i exctracted it

3) imported stopWords to remove the words in the tweet which doesn't have any contextual importance

4) loading the csv data into a panda dataframe, and saw it has 1.6 mil dataframe

5)it wasn't reading the coloumn names, so i created coloumn names and created the data frame again

6)checked the shape and any missing values in the dataframe then  distribution

7)converted al the "4" values to "1" from the target label, for my convinience

8)performed stemming operation to reduce the words to its root word

9)created a new coloumn called stemmed coloumn

10)split the dataframe into train and test datas

11)used vectorizer to convert textual data into numerical data (feature extraction)

12)trained the machine learning model using Logistic regression

13)checked the model accuracy

14)then saved the trained model using pickle with.sav extention

15) and checked indivial tweet randomly if they are positive or negative comparing with the true value

